**Introdução** Neste estudo, vamos a  determinar quais variáveis têm influência no número de streams e se a popularidade de artistas no Spotify está relacionada à popularidade em outros serviços de streaming, vamos usar técnicas de regressão, como regressão linear múltipla para prever o número de streams com base nas variáveis independentes e análise de correlação para examinar as relações entre as popularidades nos diferentes serviços de streaming..

In [ ]:
# importar pandas e tabelas
import pandas as pd
spotify = pd.read_csv('/content/track_in_spotify - spotify.csv')
competition = pd.read_csv('/content/track_in_competition - competition.csv')
technical = pd.read_csv('/content/track_technical_info - technical_info.csv')

In [ ]:
# Unir os DataFrames usando o track_id como chave de junção
merged = pd.merge(spotify, competition, on='track_id')
merged = pd.merge(merged, technical, on='track_id')


In [ ]:
# Remover linhas com valores não numéricos na coluna streams
merged = merged[pd.to_numeric(merged['streams'], errors='coerce').notnull()]

# Converter a coluna streams para int64
merged['streams'] = merged['streams'].astype(int)

# Converter a coluna deezer playlists para int64
merged['in_deezer_playlists'] = merged['in_deezer_playlists'].str.replace(',', '').astype(int)
# Verificar se a coluna deezer playlists foi convertida com sucesso
print(merged.info())


<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_id              952 non-null    object
 1   track_name            952 non-null    object
 2   artist(s)_name        952 non-null    object
 3   artist_count          952 non-null    int64 
 4   released_year         952 non-null    int64 
 5   released_month        952 non-null    int64 
 6   released_day          952 non-null    int64 
 7   in_spotify_playlists  952 non-null    int64 
 8   in_spotify_charts     952 non-null    int64 
 9   streams               952 non-null    int64 
 10  in_apple_playlists    952 non-null    int64 
 11  in_apple_charts       952 non-null    int64 
 12  in_deezer_playlists   952 non-null    int64 
 13  in_deezer_charts      952 non-null    int64 
 14  in_shazam_charts      902 non-null    object
 15  bpm                   952 non-null    int64 


<ipython-input-8-a8e133dfb1d0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['streams'] = merged['streams'].astype(int)
<ipython-input-8-a8e133dfb1d0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['in_deezer_playlists'] = merged['in_deezer_playlists'].str.replace(',', '').astype(int)


In [ ]:
# Somar as colunas de playlists
merged['total_playlists'] = merged['in_spotify_playlists'] + merged['in_apple_playlists'] + merged['in_deezer_playlists']

# Verificar se a nova coluna foi criada corretamente
print(merged.info())


<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 952
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_id              952 non-null    object
 1   track_name            952 non-null    object
 2   artist(s)_name        952 non-null    object
 3   artist_count          952 non-null    int64 
 4   released_year         952 non-null    int64 
 5   released_month        952 non-null    int64 
 6   released_day          952 non-null    int64 
 7   in_spotify_playlists  952 non-null    int64 
 8   in_spotify_charts     952 non-null    int64 
 9   streams               952 non-null    int64 
 10  in_apple_playlists    952 non-null    int64 
 11  in_apple_charts       952 non-null    int64 
 12  in_deezer_playlists   952 non-null    int64 
 13  in_deezer_charts      952 non-null    int64 
 14  in_shazam_charts      902 non-null    object
 15  bpm                   952 non-null    int64 


In [ ]:
# Remover colunas foro escopo
merged = merged.drop(['released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'in_deezer_playlists', 'in_apple_playlists', 'key', 'mode', 'in_shazam_charts'], axis=1)

# Verificar se as colunas foi eliminada com sucesso
print(merged.info())

<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 952
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   track_id            952 non-null    object
 1   track_name          952 non-null    object
 2   artist(s)_name      952 non-null    object
 3   artist_count        952 non-null    int64 
 4   in_spotify_charts   952 non-null    int64 
 5   streams             952 non-null    int64 
 6   in_apple_charts     952 non-null    int64 
 7   in_deezer_charts    952 non-null    int64 
 8   bpm                 952 non-null    int64 
 9   danceability_%      952 non-null    int64 
 10  valence_%           952 non-null    int64 
 11  energy_%            952 non-null    int64 
 12  acousticness_%      952 non-null    int64 
 13  instrumentalness_%  952 non-null    int64 
 14  liveness_%          952 non-null    int64 
 15  speechiness_%       952 non-null    int64 
 16  total_playlists     952 non-nul

Para a regressão linear ou multilinear, não é necessário que os dados sigam uma **distribuição normal**. A suposição de normalidade se aplica aos **resíduos do modelo**, não aos dados originais. Ou seja, os resíduos devem ter uma distribuição normal para que as estimativas dos parâmetros do modelo sejam não tendenciosas e eficientes.

In [ ]:
#importa o modelo regressao linear
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Hipótese 1 - Músicas com BPM mais altos fazem mais sucesso
X = merged['bpm']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_1 = model.resid
print("Hipótese 1 - Músicas com BPM mais altos fazem mais sucesso:")
print(model.summary())
print('\nResposta: ', residuos_hipotese_1)

# Hipótese 2 - Comparação entre Spotify e Deezer
X = merged['in_deezer_charts']
Y = merged['in_spotify_charts']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_deezer = model.resid
print("\nHipótese 2 - Comparação entre Spotify e Deezer:")
print(model.summary())
print('\nResposta: ', residuos_hipotese_deezer)
# Hipótese 2 - Comparação entre Spotify e Apple
X = merged['in_apple_charts']
Y = merged['in_spotify_charts']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_apple = model.resid
print("\nHipótese 2 - Comparação entre Spotify e Apple:")
print(model.summary())

# Hipótese 3 - Correlação entre playlists e streams
X = merged['total_playlists']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_3 = model.resid
print("\nHipótese 3 - Correlação entre playlists e streams:")
print(model.summary())

#hipótese 4 -  Artistas com um maior número de músicas têm mais streams
X = merged['artist_count']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_4 = model.resid
print("\nHipótese 4 - Artistas com um maior número de músicas têm mais streams:")
print(model.summary())

# Hipótese 5 - Influência da danceability no número de streams
X = merged['danceability_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_dance = model.resid
print("\nHipótese 5 - Influência da danceability no número de streams:")
print(model.summary())
# Hipótese 5 - Influência da energy no número de streams
X = merged['energy_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_energy = model.resid
print("\nHipótese 5 - Influência da energy no número de streams:")
print(model.summary())
# Hipótese 5 - Influência da valence no número de streams
X = merged['valence_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_valence = model.resid
print("\nHipótese 5 - Influência da valence no número de streams:")
print(model.summary())
# Hipótese 5 - Influência da liveness no número de streams
X = merged['liveness_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_liveness = model.resid
print("\nHipótese 5 - Influência da liveness no número de streams:")
print(model.summary())
# Hipótese 5 - Influência da instrumentalness no número de streams
X = merged['instrumentalness_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_instrumental = model.resid
print("\nHipótese 5 - Influência da instrumentalness no número de streams:")
print(model.summary())
# Hipótese 5 - Influência da acousticness no número de streams
X = merged['acousticness_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_acoutic = model.resid
print("\nHipótese 5 - Influência da acousticness no número de streams:")
print(model.summary())
# Hipótese 5 - Influência da speechiness no número de streams
X = merged['speechiness_%']
Y = merged['streams']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
# Calcular os resíduos
residuos_hipotese_speech = model.resid
print("\nHipótese 5 - Influência da speechiness no número de streams:")
print(model.summary())



Hipótese 1 - Músicas com BPM mais altos fazem mais sucesso:
                            OLS Regression Results                            
Dep. Variable:                streams   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.005646
Date:                Wed, 01 May 2024   Prob (F-statistic):              0.940
Time:                        16:46:04   Log-Likelihood:                -20538.
No. Observations:                 952   AIC:                         4.108e+04
Df Residuals:                     950   BIC:                         4.109e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [ ]:
from scipy.stats import shapiro

# Teste de Shapiro-Wilk
stat, p_valor = shapiro(residuos_hipotese_1)
# Verificar o resultado
if p_valor > 0.05:
    print("Os resíduos da Hipótese 1 parecem seguir uma distribuição normal.")
else:
    print("Os resíduos da Hipótese 1 não seguem uma distribuição normal.")

# Teste de Shapiro-Wilk
stat, p_valor = shapiro(residuos_hipotese_deezer)
# Verificar o resultado
if p_valor > 0.05:
    print("Os resíduos da Hipótese deezer parecem seguir uma distribuição normal.")
else:
    print("Os resíduos da Hipótese deezer não seguem uma distribuição normal.")

# Teste de Shapiro-Wilk
stat, p_valor = shapiro(residuos_hipotese_apple)
# Verificar o resultado
if p_valor > 0.05:
    print("Os resíduos da Hipótese apple parecem seguir uma distribuição normal.")
else:
    print("Os resíduos da Hipótese apple não seguem uma distribuição normal.")

# Teste de Shapiro-Wilk
stat, p_valor = shapiro(residuos_hipotese_3)
# Verificar o resultado
if p_valor > 0.05:
    print("Os resíduos da Hipótese 3 parecem seguir uma distribuição normal.")
else:
    print("Os resíduos da Hipótese 3 não seguem uma distribuição normal.")

# Teste de Shapiro-Wilk
stat, p_valor = shapiro(residuos_hipotese_4)
# Verificar o resultado
if p_valor > 0.05:
    print("Os resíduos da Hipótese 4 parecem seguir uma distribuição normal.")
else:
    print("Os resíduos da Hipótese 4 não seguem uma distribuição normal.")




Os resíduos da Hipótese 1 não seguem uma distribuição normal.
Os resíduos da Hipótese deezer não seguem uma distribuição normal.
Os resíduos da Hipótese apple não seguem uma distribuição normal.
Os resíduos da Hipótese 3 não seguem uma distribuição normal.
Os resíduos da Hipótese 4 não seguem uma distribuição normal.


**A Regressão Linear Múltipla** é um modelo de análise que usamos quando modelamos a relação linear entre uma variável de desfecho(dependente) contínua e múltiplas variáveis preditoras(independente) que podem ser contínuas ou categóricas.

In [ ]:
# Importando as bibliotecas
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

# os dados, onde X é independente e Y dependente
X = merged[['total_playlists', 'in_spotify_charts', 'bpm', 'danceability_%', 'energy_%', 'instrumentalness_%', 'valence_%', 'acousticness_%', 'liveness_%', 'speechiness_%', 'in_deezer_charts', 'in_apple_charts']]
y = merged['streams']

# Criando e treinando o modelo de Regressão Linear Múltipla
model_multiplo = LinearRegression()
model_multiplo.fit(X, y)

# Coeficientes do modelo
coeficientes = pd.DataFrame(model_multiplo.coef_, X.columns, columns=['Coeficiente'])
print("Coeficientes do modelo linear multiplo:")
print(coeficientes)

# Calculando a correlação entre as variáveis
correlacao_spotify_deezer, _ = pearsonr(merged['in_spotify_charts'], merged['in_deezer_charts'])
correlacao_spotify_apple, _ = pearsonr(merged['in_spotify_charts'], merged['in_apple_charts'])
print(f"\nCorrelação entre popularidade no Spotify e no Deezer: {correlacao_spotify_deezer}")
print(f"\nCorrelação entre popularidade no Spotify e no Apple Music: {correlacao_spotify_apple}")


Coeficientes do modelo linear multiplo:
                     Coeficiente
total_playlists     4.739054e+04
in_spotify_charts   1.371330e+06
bpm                 2.087332e+05
danceability_%     -2.784408e+05
energy_%           -1.894070e+06
instrumentalness_% -2.114212e+06
valence_%          -3.746415e+05
acousticness_%      4.522268e+05
liveness_%         -2.133940e+05
speechiness_%      -1.516949e+06
in_deezer_charts    6.972174e+06
in_apple_charts     7.862633e+05

Correlação entre popularidade no Spotify e no Deezer: 0.6001007474789155

Correlação entre popularidade no Spotify e no Apple Music: 0.551985326175484


In [ ]:
from scipy.stats import shapiro

# Calculando os resíduos do modelo
merged['residuos'] = y - model_multiplo.predict(X)

# Teste de Shapiro-Wilk para verificar a normalidade dos resíduos
stat, p_valor = shapiro(merged['residuos'])

# Verificando o resultado do teste
if p_valor > 0.05:
    print("Os resíduos parecem ser normalmente distribuídos.")
else:
    print("Os resíduos não parecem ser normalmente distribuídos.")


Os resíduos não parecem ser normalmente distribuídos.


**Quando os resíduos do seu modelo de Regressão Linear Múltipla não têm uma distribuição normal, há algumas opções que você pode considerar:**
1- **Transformações nos dados:** Experimente transformar suas variáveis ou a variável de resposta para tornar os resíduos mais normalmente distribuídos. Por exemplo, você pode tentar transformações logarítmicas ou raiz quadrada.
2- **Modelos alternativos:** Considere o uso de modelos diferentes que sejam mais adequados para seus dados. Por exemplo, modelos de regressão robusta podem ser mais apropriados para dados com resíduos não normalmente distribuídos.
3- **Considerar outras variáveis:** Pode haver variáveis importantes que não foram incluídas no modelo e que podem ajudar a explicar a variabilidade nos dados.
4- **Validação do modelo:** Verifique se o modelo está se ajustando bem aos dados de treinamento e se é capaz de fazer previsões precisas em dados de teste.
5- **Análise adicional:** Realize uma análise mais aprofundada dos resíduos, como a análise de resíduos versus previsões ou resíduos versus variáveis independentes, para identificar padrões ou tendências que possam indicar problemas com o modelo.

In [ ]:
# Tranformaçao nos dados
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro

# Normalizando os dados
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Criando e treinando o modelo de Regressão Linear Múltipla com dados normalizados
model_multiplo = LinearRegression()
model_multiplo.fit(X_scaled, y_scaled)

# Coeficientes do modelo
coeficientes = pd.DataFrame(model_multiplo.coef_.reshape(-1, 1), X.columns, columns=['Coeficiente'])
print("Coeficientes do modelo:")
print(coeficientes)

# Verificando a distribuição dos resíduos
residuos = y_scaled - model_multiplo.predict(X_scaled)
stat, p_valor = shapiro(residuos)
if p_valor > 0.05:
    print("\nOs resíduos parecem ser normalmente distribuídos após a normalização dos dados.")
else:
    print("\nOs resíduos ainda não parecem ser normalmente distribuídos após a normalização dos dados.")


Coeficientes do modelo:
                    Coeficiente
total_playlists        0.745656
in_spotify_charts      0.047373
bpm                    0.010336
danceability_%        -0.007187
energy_%              -0.055328
instrumentalness_%    -0.031382
valence_%             -0.015519
acousticness_%         0.020744
liveness_%            -0.005164
speechiness_%         -0.026534
in_deezer_charts       0.074267
in_apple_charts        0.070225

Os resíduos ainda não parecem ser normalmente distribuídos após a normalização dos dados.


In [ ]:
# Modelos alternativos: modelo de regressao robusta
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import shapiro

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Criando e ajustando o modelo de regressão robusta
model_robusto = HuberRegressor()
model_robusto.fit(X_scaled, y)

# Coeficientes do modelo robusto
coeficientes_robustos = pd.DataFrame(model_robusto.coef_.reshape(-1, 1), X.columns, columns=['Coeficiente Robusto'])
print("\nCoeficientes do modelo robusto:")
print(coeficientes_robustos)

# Calculando os resíduos do modelo
residuos = y - model_robusto.predict(X_scaled)

# Teste de Shapiro-Wilk para verificar a normalidade dos resíduos
stat, p_valor = shapiro(residuos)

# Verificando o resultado do teste
if p_valor > 0.05:
    print("\nOs resíduos parecem ser normalmente distribuídos.")
else:
    print("\nOs resíduos não parecem ser normalmente distribuídos.")



Coeficientes do modelo robusto:
                    Coeficiente Robusto
total_playlists            5.790611e+06
in_spotify_charts          1.746301e+06
bpm                        2.394351e+05
danceability_%            -8.212929e+05
energy_%                  -2.529811e+05
instrumentalness_%        -1.717155e+05
valence_%                 -4.539857e+05
acousticness_%            -3.995358e+05
liveness_%                -4.510277e+05
speechiness_%             -1.138853e+06
in_deezer_charts           2.054286e+06
in_apple_charts            2.784535e+06

Os resíduos não parecem ser normalmente distribuídos.


Quanto à **normalidade dos resíduos**, o teste de Shapiro-Wilk sugere que os resíduos **não seguem uma distribuição normal**, mesmo após a normalização dos dados. Isso pode indicar que o **modelo não captura** completamente a relação entre as variáveis independentes e dependentes, ou pode haver **outras variáveis importantes não incluídas no modelo**.

Nesse caso, pode ser útil explorar outras técnicas de modelagem(fiz robusta) ou considerar a inclusão de outras variáveis(nao tenho) no modelo para melhorar a distribuição dos resíduos.

In [ ]:
#Validaçao do Modelo
#importar biblio
from sklearn.metrics import mean_absolute_error
import numpy as np

# Calculando os resíduos
residuos = y_scaled - model_multiplo.predict(X_scaled)

# Calculando as métricas
mae = mean_absolute_error(y_scaled, model_multiplo.predict(X_scaled))
mape = np.mean(np.abs(residuos) / np.abs(y_scaled)) * 100
median_residuos = np.median(residuos)

print(f"Erro Absoluto Médio (MAE): {mae}") #o modelo erra cerca de 0.3879 unidades ao prever os dados normalizados.
print(f"Erro Percentual Absoluto Médio (MAPE): {mape}") #19.36%. Isso indica um erro percentual considerável, mostrando que o modelo tem dificuldade em prever com precisão os dados normalizados.
print(f"Mediana dos Resíduos: {median_residuos}") #-0.0991. Isso significa que a metade dos resíduos é maior que esse valor e a outra metade é menor.


Erro Absoluto Médio (MAE): 0.3878992659093228
Erro Percentual Absoluto Médio (MAPE): 119.36087129509222
Mediana dos Resíduos: -0.09910663451673296


In [ ]:
#Validaçao do Modelo
#importar biblio
from sklearn.metrics import mean_absolute_error
import numpy as np

# Calculando os resíduos
residuos = y_scaled - model_robusto.predict(X_scaled)

# Calculando as métricas
mae = mean_absolute_error(y_scaled, model_robusto.predict(X_scaled))
mape = np.mean(np.abs(residuos) / np.abs(y_scaled)) * 100
median_residuos = np.median(residuos)

print(f"Erro Absoluto Médio (MAE): {mae}") #Isso significa que, em média, o modelo erra cerca de 347.8 milhões de unidades ao prever o número de streams.
print(f"Erro Percentual Absoluto Médio (MAPE): {mape}") #174400888118.3%, o que indica um erro percentual extremamente alto. Isso sugere que o modelo tem dificuldade em prever os dados com precisão.
print(f"Mediana dos Resíduos: {median_residuos}") #-345.1 milhões. Isso significa que a metade dos resíduos é maior que esse valor e a outra metade é menor

Erro Absoluto Médio (MAE): 347798671.0215691
Erro Percentual Absoluto Médio (MAPE): 174400888118.31088
Mediana dos Resíduos: -345138608.00053984


In [ ]:
#importa o modelo
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Hipótese 1 - Músicas com BPM mais altos fazem mais sucesso
X = merged[['bpm']]
Y = merged['streams']
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Adiciona uma constante para o termo independente
X_scaled = sm.add_constant(X_scaled)
model = sm.OLS(Y, X_scaled).fit()
# Calcular os resíduos
residuos_hipotese_1 = model.resid
print("Hipótese 1 - Músicas com BPM mais altos fazem mais sucesso:")
print(model.summary())
print('\nResposta: ', residuos_hipotese_1)

# Hipótese 2 - Comparação entre Spotify e Deezer
X = merged[['in_deezer_charts']]
Y = merged['in_spotify_charts']
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Adiciona uma constante para o termo independente
X_scaled = sm.add_constant(X_scaled)
model = sm.OLS(Y, X_scaled).fit()
# Calcular os resíduos
residuos_hipotese_deezer = model.resid
print("\nHipótese 2 - Comparação entre Spotify e Deezer:")
print(model.summary())
print('\nResposta: ', residuos_hipotese_deezer)
# Hipótese 2 - Comparação entre Spotify e Apple
X = merged[['in_apple_charts']]
Y = merged['in_spotify_charts']
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Adiciona uma constante para o termo independente
X_scaled = sm.add_constant(X_scaled)
model = sm.OLS(Y, X_scaled).fit()
# Calcular os resíduos
residuos_hipotese_apple = model.resid
print("\nHipótese 2 - Comparação entre Spotify e Apple:")
print(model.summary())

# Hipótese 3 - Correlação entre playlists e streams
X = merged[['total_playlists']]
Y = merged['streams']
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Adiciona uma constante para o termo independente
X_scaled = sm.add_constant(X_scaled)
model = sm.OLS(Y, X_scaled).fit()
# Calcular os resíduos
residuos_hipotese_3 = model.resid
print("\nHipótese 3 - Correlação entre playlists e streams:")
print(model.summary())

#hipótese 4 -  Artistas com um maior número de músicas têm mais streams
X = merged[['artist_count']]
Y = merged['streams']
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Adiciona uma constante para o termo independente
X_scaled = sm.add_constant(X_scaled)
model = sm.OLS(Y, X_scaled).fit()
# Calcular os resíduos
residuos_hipotese_4 = model.resid
print("\nHipótese 4 - Artistas com um maior número de músicas têm mais streams:")
print(model.summary())

# Hipótese 5 - Influência da danceability no número de streams
X = merged[['danceability_%']]
Y = merged['streams']
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Adiciona uma constante para o termo independente
X_scaled = sm.add_constant(X_scaled)
model = sm.OLS(Y, X_scaled).fit()
# Calcular os resíduos
residuos_hipotese_dance = model.resid
print("\nHipótese 5 - Influência da danceability no número de streams:")
print(model.summary())

Hipótese 1 - Músicas com BPM mais altos fazem mais sucesso:
                            OLS Regression Results                            
Dep. Variable:                streams   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.005646
Date:                Wed, 01 May 2024   Prob (F-statistic):              0.940
Time:                        16:47:03   Log-Likelihood:                -20538.
No. Observations:                 952   AIC:                         4.108e+04
Df Residuals:                     950   BIC:                         4.109e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [ ]:
from scipy.stats import shapiro

# Realizar o teste de Shapiro-Wilk para verificar a normalidade dos resíduos
shapiro_test_h1 = shapiro(residuos_hipotese_1)
shapiro_test_deezer = shapiro(residuos_hipotese_deezer)
shapiro_test_apple = shapiro(residuos_hipotese_apple)
shapiro_test_3 = shapiro(residuos_hipotese_3)
shapiro_test_4 = shapiro(residuos_hipotese_4)
shapiro_test_dance = shapiro(residuos_hipotese_dance)

# Exibir os resultados dos testes de normalidade
print("Teste de Shapiro-Wilk para normalidade dos resíduos:")
print("\nHipótese 1: ", shapiro_test_h1)
print("\nHipótese 2 (Deezer):", shapiro_test_deezer)
print("\nHipótese 2 (Apple):", shapiro_test_apple)
print("\nHipótese 3:", shapiro_test_3)
print("\nHipótese 4:", shapiro_test_4)
print("\nHipótese 5 (Danceability):", shapiro_test_dance)


Teste de Shapiro-Wilk para normalidade dos resíduos:

Hipótese 1:  ShapiroResult(statistic=0.7615191340446472, pvalue=5.839652174878024e-35)

Hipótese 2 (Deezer): ShapiroResult(statistic=0.7827615141868591, pvalue=9.830314872816198e-34)

Hipótese 2 (Apple): ShapiroResult(statistic=0.8743630647659302, pvalue=5.340751647205534e-27)

Hipótese 3: ShapiroResult(statistic=0.8318015933036804, pvalue=1.7282454607371346e-30)

Hipótese 4: ShapiroResult(statistic=0.7779949307441711, pvalue=5.120846448105441e-34)

Hipótese 5 (Danceability): ShapiroResult(statistic=0.7787995338439941, pvalue=5.7122922981243474e-34)


In [ ]:
#Validaçao do Modelo
#importar biblio
from sklearn.metrics import mean_absolute_error
import numpy as np

# Calculando os resíduos
residuos = y_scaled - model.predict(X_scaled)

# Calculando as métricas
mae = mean_absolute_error(y_scaled, model.predict(X_scaled))
mape = np.mean(np.abs(residuos) / np.abs(y_scaled)) * 100
median_residuos = np.median(residuos)

print(f"Erro Absoluto Médio (MAE): {mae}") #514.1 milhões. Isso significa que, em média, o modelo erra cerca de 514.1 milhões de unidades ao prever o número de streams.
print(f"Erro Percentual Absoluto Médio (MAPE): {mape}") #aproximadamente 257810138419.3%, o que indica um erro percentual extremamente alto. Isso sugere que o modelo tem dificuldade em prever os dados com precisão.
print(f"Mediana dos Resíduos: {median_residuos}") #-505.9 milhões. Isso significa que a metade dos resíduos é maior que esse valor e a outra metade é menor.

Erro Absoluto Médio (MAE): 514137424.93907565
Erro Percentual Absoluto Médio (MAPE): 257810138419.30414
Mediana dos Resíduos: -505901658.02224904


Os **resultados** dos testes indicam que os modelos **pode não estar performando bem na previsão dos dados**. O MAE e o MAPE indicam que há uma diferença significativa entre os valores previstos e os valores reais. Além disso, a mediana dos resíduos sugere que o modelo pode estar subestimando os valores em geral. Portanto, seria recomendável investigar e possivelmente ajustar o modelo para melhorar sua precisão.


**O teste de Wald** é comumente usado para testar a **significância de um coeficiente em um modelo de regressão**. Ele não determina **causalidade diretamente**, mas pode indicar se há uma **relação significativa entre uma variável independente e a variável dependente no modelo**. A causalidade requer mais do que simplesmente testar a significância de um coeficiente; geralmente requer uma análise cuidadosa, incluindo a consideração de outros fatores e possíveis viéses.

In [ ]:
#Não é necessário normalizar os dados para realizar os testes de Wald, Granger
#ou Sims. Esses testes são baseados nos coeficientes estimados dos modelos e nas propriedades estatísticas dos resíduos, não nos valores brutos das variáveis. (pode realizar esses testes com os dados originais).

import pandas as pd
import statsmodels.api as sm
from scipy.stats import chi2

# Seus dados
X = merged[['total_playlists', 'in_spotify_charts', 'bpm', 'danceability_%']]
y = merged['streams']

# Adiciona uma constante para o termo independente
X = sm.add_constant(X)

# Ajusta o modelo
model_wald = sm.OLS(y, X).fit()

# Matriz de restrição para testar todos os coeficientes simultaneamente
r_matrix = np.eye(len(model_wald.params))

# Realiza o teste de Wald
wald_test = model_wald.wald_test(r_matrix)

# Obtém o valor crítico para o teste qui-quadrado
alpha = 0.05
df = len(model_wald.params)  # Graus de liberdade
critical_value = chi2.ppf(1 - alpha, df)

# Imprime o resultado do teste
print('Valor do teste de Wald:', wald_test.statistic[0][0])
print('Valor crítico:', critical_value)
print('P-valor:', wald_test.pvalue)
print('Os coeficientes são significativos?', wald_test.statistic[0][0] > critical_value)


Valor do teste de Wald: 742.010611383438
Valor crítico: 11.070497693516351
P-valor: 0.0
Os coeficientes são significativos? True


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


Utilizar **modelos de séries temporais** para analisar a relação entre as variáveis ao longo do tempo e identificar possíveis relações causais.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import shapiro

X = merged[['total_playlists', 'in_spotify_charts', 'bpm', 'danceability_%']]
Y = merged['streams']

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Criando o modelo de série temporal com os dados normalizados
model_tempo = sm.tsa.ARIMA(endog=Y, exog=X_scaled, order=(1, 0, 0))
results = model_tempo.fit()

# Calculando as métricas
log_likelihood = results.llf
aic = results.aic
bic = results.bic
mae = np.mean(np.abs(results.resid))
mape = np.mean(np.abs(results.resid) / np.abs(Y))
ljung_box = sm.stats.acorr_ljungbox(results.resid, lags=[10], return_df=True).iloc[0, 1]
jarque_bera = sm.stats.jarque_bera(results.resid)[0]
shapiro_test = shapiro(results.resid)

# Exibindo as métricas
print(f"Log Likelihood: {log_likelihood}") #medida da adequação do modelo aos dados. Quanto maior, melhor.
print(f"\nAIC: {aic}") #medida da qualidade relativa de um modelo estatístico, levando em conta a complexidade do modelo. Quanto menor, melhor.
print(f"\nBIC: {bic}") #É similar ao AIC, mas penaliza modelos com mais parâmetros de forma mais severa. Quanto menor, melhor.
print(f"\nMAE: {mae}") #É a média dos valores absolutos dos resíduos. Quanto menor, melhor.
print(f"\nMAPE: {mape}") #É a média da porcentagem dos valores absolutos dos resíduos em relação aos valores reais. Quanto menor, melhor.
print(f"\nLjung-Box (10 lags): {ljung_box}") # teste de autocorrelação dos resíduos. Um valor baixo indica que os resíduos não têm autocorrelação.
print(f"\nJarque-Bera: {jarque_bera}") #teste de normalidade dos resíduos. Quanto maior, mais diferente os resíduos são da distribuição normal.
print(f"\nShapiro-Wilk Test (Statistic, p-value): {shapiro_test}")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Log Likelihood: -20056.235677295583

AIC: 40126.471354591165

BIC: 40160.48130983471

MAE: 221865448.51397413

MAPE: 132.8034480529114

Ljung-Box (10 lags): 0.0036304283571794754

Jarque-Bera: 1479.0021053385244

Shapiro-Wilk Test (Statistic, p-value): ShapiroResult(statistic=0.858115553855896, pvalue=1.995314098416838e-28)


Com base nessas métricas, o modelo ARIMA parece **ter um bom ajuste aos dados**, com **exceção do teste de normalidade dos resíduos**, que sugere que os resíduos **não são normalmente distribuídos**. Isso pode indicar que o modelo não captura completamente a estrutura dos dados ou que há algum padrão nos resíduos que não foi capturado pelo modelo.

O **teste de causalidade de Granger** é uma ferramenta útil para investigar a relação de causa e efeito entre variáveis em séries temporais, ajudando a identificar quais variáveis são importantes para prever o comportamento de outras variáveis ao longo do tempo.

In [ ]:
# Importar o teste de causalidade de Granger
from statsmodels.tsa.stattools import grangercausalitytests

# Selecionar as variáveis para o teste de causalidade de Granger
data = merged[['total_playlists', 'streams']]

# Definir o número máximo de lags a serem considerados
max_lag = 3

# Executar o teste de causalidade de Granger
results_granger = grangercausalitytests(data, max_lag, verbose=True)

# Verificar se o valor-p é menor que 0.05
for lag in range(1, max_lag + 1):
    p_value = results_granger[lag][0]['ssr_ftest'][1]
    if p_value < 0.05:
        print(f'O teste de causalidade de Granger para lag {lag} é significativo (p-valor = {p_value}).')
    else:
        print(f'O teste de causalidade de Granger para lag {lag} não é significativo (p-valor = {p_value}).')

# Exibir os resultados completos do teste de causalidade de Granger
print('\nResultados completos do teste de Causalidade de Granger:')
print(results_granger)




Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2922 , p=0.0005  , df_denom=948, df_num=1
ssr based chi2 test:   chi2=12.3311 , p=0.0004  , df=1
likelihood ratio test: chi2=12.2518 , p=0.0005  , df=1
parameter F test:         F=12.2922 , p=0.0005  , df_denom=948, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=10.7802 , p=0.0000  , df_denom=945, df_num=2
ssr based chi2 test:   chi2=21.6744 , p=0.0000  , df=2
likelihood ratio test: chi2=21.4309 , p=0.0000  , df=2
parameter F test:         F=10.7802 , p=0.0000  , df_denom=945, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=11.4847 , p=0.0000  , df_denom=942, df_num=3
ssr based chi2 test:   chi2=34.7101 , p=0.0000  , df=3
likelihood ratio test: chi2=34.0904 , p=0.0000  , df=3
parameter F test:         F=11.4847 , p=0.0000  , df_denom=942, df_num=3
O teste de causalidade de Granger para lag 1 é significativo (p-valor = 0.0004761743060247488)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Preparar os dados
data = merged[['in_spotify_charts', 'streams']]
max_lag = 3  # Defina o número máximo de lags a serem considerados

# Executar o teste de causalidade de Granger
results_granger = grangercausalitytests(data, max_lag, verbose=True)

# Avaliar os resultados
for lag in results_granger.keys():
    causality = results_granger[lag][0]['ssr_ftest'][1]
    if causality < 0.05:  # Verifique se a causalidade é estatisticamente significativa
        print(f'O teste de causalidade de Granger para lag {lag} é significativo (p-valor = {p_value}).')
    else:
        print(f'O teste de causalidade de Granger para lag {lag} não é significativo (p-valor = {p_value}).')

# Exibir os resultados completos do teste de causalidade de Granger
print('\nResultados completos do teste de Causalidade de Granger:')
print(results_granger)



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.8057  , p=0.0053  , df_denom=948, df_num=1
ssr based chi2 test:   chi2=7.8304  , p=0.0051  , df=1
likelihood ratio test: chi2=7.7983  , p=0.0052  , df=1
parameter F test:         F=7.8057  , p=0.0053  , df_denom=948, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.8651  , p=0.0213  , df_denom=945, df_num=2
ssr based chi2 test:   chi2=7.7711  , p=0.0205  , df=2
likelihood ratio test: chi2=7.7395  , p=0.0209  , df=2
parameter F test:         F=3.8651  , p=0.0213  , df_denom=945, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4447  , p=0.2283  , df_denom=942, df_num=3
ssr based chi2 test:   chi2=4.3662  , p=0.2245  , df=3
likelihood ratio test: chi2=4.3562  , p=0.2255  , df=3
parameter F test:         F=1.4447  , p=0.2283  , df_denom=942, df_num=3
O teste de causalidade de Granger para lag 1 é significativo (p-valor = 2.1239823414264447e-07

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Preparar os dados
data = merged[['bpm', 'streams']]
# Executar o teste de causalidade de Granger
max_lag = 3  # Defina o número máximo de lags a serem considerados
results_granger = grangercausalitytests(data, max_lag, verbose=True)

# Avaliar os resultados
for lag in results_granger.keys():
    causality = results_granger[lag][0]['ssr_ftest'][1]
    if causality < 0.05:  # Verifique se a causalidade é estatisticamente significativa
       print(f'O teste de causalidade de Granger para lag {lag} é significativo (p-valor = {p_value}).')
    else:
        print(f'O teste de causalidade de Granger para lag {lag} não é significativo (p-valor = {p_value}).')

# Exibir os resultados completos do teste de causalidade de Granger
print('\nResultados completos do teste de Causalidade de Granger:')
print(results_granger)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.7628  , p=0.3827  , df_denom=948, df_num=1
ssr based chi2 test:   chi2=0.7652  , p=0.3817  , df=1
likelihood ratio test: chi2=0.7649  , p=0.3818  , df=1
parameter F test:         F=0.7628  , p=0.3827  , df_denom=948, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4217  , p=0.6561  , df_denom=945, df_num=2
ssr based chi2 test:   chi2=0.8478  , p=0.6545  , df=2
likelihood ratio test: chi2=0.8474  , p=0.6546  , df=2
parameter F test:         F=0.4217  , p=0.6561  , df_denom=945, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3046  , p=0.8221  , df_denom=942, df_num=3
ssr based chi2 test:   chi2=0.9206  , p=0.8205  , df=3
likelihood ratio test: chi2=0.9201  , p=0.8206  , df=3
parameter F test:         F=0.3046  , p=0.8221  , df_denom=942, df_num=3
O teste de causalidade de Granger para lag 1 não é significativo (p-valor = 2.1239823414264447

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Preparar os dados
data = merged[['danceability_%', 'streams']]
# Executar o teste de causalidade de Granger
max_lag = 3  # Defina o número máximo de lags a serem considerados
results_granger = grangercausalitytests(data, max_lag, verbose=True)

# Avaliar os resultados
for lag in results_granger.keys():
    causality = results_granger[lag][0]['ssr_ftest'][1]
    if causality < 0.05:  # Verifique se a causalidade é estatisticamente significativa
       print(f'O teste de causalidade de Granger para lag {lag} é significativo (p-valor = {p_value}).')
    else:
        print(f'O teste de causalidade de Granger para lag {lag} não é significativo (p-valor = {p_value}).')

# Exibir os resultados completos do teste de causalidade de Granger
print('\nResultados completos do teste de Causalidade de Granger:')
print(results_granger)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=4.7500  , p=0.0295  , df_denom=948, df_num=1
ssr based chi2 test:   chi2=4.7651  , p=0.0290  , df=1
likelihood ratio test: chi2=4.7532  , p=0.0292  , df=1
parameter F test:         F=4.7500  , p=0.0295  , df_denom=948, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.4030  , p=0.0910  , df_denom=945, df_num=2
ssr based chi2 test:   chi2=4.8314  , p=0.0893  , df=2
likelihood ratio test: chi2=4.8191  , p=0.0899  , df=2
parameter F test:         F=2.4030  , p=0.0910  , df_denom=945, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.8362  , p=0.1389  , df_denom=942, df_num=3
ssr based chi2 test:   chi2=5.5495  , p=0.1357  , df=3
likelihood ratio test: chi2=5.5333  , p=0.1367  , df=3
parameter F test:         F=1.8362  , p=0.1389  , df_denom=942, df_num=3
O teste de causalidade de Granger para lag 1 é significativo (p-valor = 2.1239823414264447e-07

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Preparar os dados
data = merged[['liveness_%', 'streams']]
# Executar o teste de causalidade de Granger
max_lag = 3  # Defina o número máximo de lags a serem considerados
results_granger = grangercausalitytests(data, max_lag, verbose=True)

# Avaliar os resultados
for lag in results_granger.keys():
    causality = results_granger[lag][0]['ssr_ftest'][1]
    if causality < 0.05:  # Verifique se a causalidade é estatisticamente significativa
       print(f'O teste de causalidade de Granger para lag {lag} é significativo (p-valor = {p_value}).')
    else:
        print(f'O teste de causalidade de Granger para lag {lag} não é significativo (p-valor = {p_value}).')

# Exibir os resultados completos do teste de causalidade de Granger
print('\nResultados completos do teste de Causalidade de Granger:')
print(results_granger)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.0583  , p=0.0247  , df_denom=948, df_num=1
ssr based chi2 test:   chi2=5.0744  , p=0.0243  , df=1
likelihood ratio test: chi2=5.0609  , p=0.0245  , df=1
parameter F test:         F=5.0583  , p=0.0247  , df_denom=948, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.5697  , p=0.0771  , df_denom=945, df_num=2
ssr based chi2 test:   chi2=5.1665  , p=0.0755  , df=2
likelihood ratio test: chi2=5.1525  , p=0.0761  , df=2
parameter F test:         F=2.5697  , p=0.0771  , df_denom=945, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.7620  , p=0.1528  , df_denom=942, df_num=3
ssr based chi2 test:   chi2=5.3254  , p=0.1495  , df=3
likelihood ratio test: chi2=5.3105  , p=0.1504  , df=3
parameter F test:         F=1.7620  , p=0.1528  , df_denom=942, df_num=3
O teste de causalidade de Granger para lag 1 é significativo (p-valor = 2.1239823414264447e-07

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Preparar os dados
data = merged[['speechiness_%', 'streams']]
# Executar o teste de causalidade de Granger
max_lag = 3  # Defina o número máximo de lags a serem considerados
results_granger = grangercausalitytests(data, max_lag, verbose=True)

# Avaliar os resultados
for lag in results_granger.keys():
    causality = results_granger[lag][0]['ssr_ftest'][1]
    if causality < 0.05:  # Verifique se a causalidade é estatisticamente significativa
       print(f'O teste de causalidade de Granger para lag {lag} é significativo (p-valor = {p_value}).')
    else:
        print(f'O teste de causalidade de Granger para lag {lag} não é significativo (p-valor = {p_value}).')

# Exibir os resultados completos do teste de causalidade de Granger
print('\nResultados completos do teste de Causalidade de Granger:')
print(results_granger)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.0033  , p=0.1573  , df_denom=948, df_num=1
ssr based chi2 test:   chi2=2.0096  , p=0.1563  , df=1
likelihood ratio test: chi2=2.0075  , p=0.1565  , df=1
parameter F test:         F=2.0033  , p=0.1573  , df_denom=948, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.2005  , p=0.3015  , df_denom=945, df_num=2
ssr based chi2 test:   chi2=2.4137  , p=0.2991  , df=2
likelihood ratio test: chi2=2.4106  , p=0.2996  , df=2
parameter F test:         F=1.2005  , p=0.3015  , df_denom=945, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.3486  , p=0.2573  , df_denom=942, df_num=3
ssr based chi2 test:   chi2=4.0757  , p=0.2534  , df=3
likelihood ratio test: chi2=4.0670  , p=0.2543  , df=3
parameter F test:         F=1.3486  , p=0.2573  , df_denom=942, df_num=3
O teste de causalidade de Granger para lag 1 não é significativo (p-valor = 2.1239823414264447

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


Com base nos resultados apresentados, **há evidência de causalidade de Granger entre as variáveis**(popularidade e playlists). Isso significa que **uma variável é útil na previsão da outra variável**, indicando uma relação causal entre elas.